## Train Othello-GPT and save to `ckpts`

Use `jupyter nbconvert --execute --to notebook --allow-errors --ExecutePreprocessor.timeout=-1 train_gpt_othello.ipynb --inplace --output ckpts/checkpoint.ipynb` to run in background

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [3]:
import os
import math
import time
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.nn import functional as F
from data import get_othello
from data.othello import permit, start_hands, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig

In [4]:
synthetic_or_championship = False  # True for training on the synthetic dataset

In [5]:
# We run get_othello with data_root = "data/othello_championship/"
othello = get_othello(ood_num=-1, data_root=None if synthetic_or_championship else "data/othello_championship", wthor=True)
train_dataset = CharDataset(othello)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=2, n_head=8, n_embd=80)
model = GPT(mconf)

Loaded 326/327 (qualified/total) sequences from liveothello2010.pgn
Loaded 303/303 (qualified/total) sequences from liveothello2011.pgn
Loaded 674/674 (qualified/total) sequences from liveothello2012.pgn
Loaded 413/413 (qualified/total) sequences from liveothello2013.pgn
Loaded 406/407 (qualified/total) sequences from liveothello2014.pgn
Loaded 487/487 (qualified/total) sequences from liveothello2015.pgn
Loaded 644/645 (qualified/total) sequences from liveothello2016.pgn
Loaded 892/893 (qualified/total) sequences from liveothello2017.pgn
Loaded 850/850 (qualified/total) sequences from liveothello2018.pgn
Loaded 950/950 (qualified/total) sequences from liveothello2019.pgn
Loaded 862/862 (qualified/total) sequences from liveothello2020.pgn
Loaded 465/465 (qualified/total) sequences from liveothello2021.pgn
Loaded 12/12 (qualified/total) sequences from WTHOR-1977.pgn
Loaded 8/8 (qualified/total) sequences from WTHOR-1978.pgn
Loaded 11/11 (qualified/total) sequences from WTHOR-1979.pgn
Loa

In [6]:
# max_epochs = 0
# # initialize a trainer instance and kick off training
# t_start = time.strftime("_%Y%m%d_%H%M%S")
# tconf = TrainerConfig(
#     max_epochs=max_epochs, 
#     batch_size=10*1,  # assuming 8 GPU's
#     learning_rate=5e-4,
#     lr_decay=True, 
#     warmup_tokens=len(train_dataset)*train_dataset.block_size*5, 
#     final_tokens=len(train_dataset)*train_dataset.block_size*max_epochs,
#     num_workers=0, 
#     ckpt_path=f"./ckpts/gpt_at{t_start}.ckpt", 
# )
# trainer = Trainer(model, train_dataset, None, tconf)
# device = trainer.device
# print(t_start)
# trainer.train()

## Or load trained model from `ckpts`

In [7]:
# load_res = model.load_state_dict(torch.load("./ckpts/gpt_synthetic.ckpt" if synthetic_or_championship else "./ckpts/gpt_championship.ckpt"))
# if torch.cuda.is_available():
#     device = torch.cuda.current_device()
#     model = model.to(device)

In [8]:
load_res = model.load_state_dict(torch.load("./ckpts/gpt_at_20230529_101657.ckpt"))
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model = model.to(device)

## Validate it: for what percentage of all partial games in validation set, the top-1 prediction is legal

In [9]:
# othello = get_othello(ood_num=1, data_root=None, wthor=True)

In [10]:
# len(othello.sequences)

In [11]:
if not synthetic_or_championship:  # for GPT trained on both datasets, use the validation set of synthetic for validation
    othello = get_othello(ood_num=-1, data_root=None, wthor=True)

Loading synthetic dataset.


Mem Used: 4.117 GB: 100%|██████████| 64/64 [00:19<00:00,  3.25it/s]


Deduplicating...
Deduplicating finished with 6399176 games left
Using 800 for training, 6398376 for validation


In [12]:
# othello = get_othello(ood_num=1, data_root=None, wthor=True, train_size=0)

In [13]:
total_nodes = 0
success_nodes = 0

bar = tqdm(othello.val[:1000])
for whole_game in bar:
    length_of_whole_game = len(whole_game)
    # print("WHOLE GAME: ", whole_game, "\n")
    for length_of_partial_game in range(1, length_of_whole_game):
        total_nodes += 1
        context = whole_game[:length_of_partial_game]
        # print("CONTEXT:", context)
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(device)
        y = sample(model, x, 1, temperature=1.0)[0]
        # print("Using {}. Prediction: {}".format(x, y))
        completion = [train_dataset.itos[int(i)] for i in y if i != -1]
        # print()
        try:
            OthelloBoardState().update(completion, prt=False)
        except Exception:
#             fail_nodes.append([permit_reverse(_) for _ in context])
            pass
        else:
            success_nodes += 1
    bar.set_description(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")
# # print(total_nodes)
print(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")

90.04% pass rate: 53111/58989 among all searched nodes: 100%|██████████| 1000/1000 [06:44<00:00,  2.47it/s]

90.04% pass rate: 53111/58989 among all searched nodes


In [14]:
1 - success_nodes/total_nodes

0.09964569665530865